<a href="https://colab.research.google.com/github/Aesol-Ahn/DA_Exercise/blob/master/Wellness_Chatbot_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.9MB/s 
     |████████████████████████████████| 460kB 43.3MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re

from konlpy.tag import Okt

In [4]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [5]:
chatbot_data = pd.read_csv("/content/gdrive/Shareddrives/파이널프로젝트/데이터/챗봇 데이터/wellness(전체).txt")
chatbot_data

,구분,유저,챗봇,category
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,0
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,0
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",0
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,0
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,0
...,...,...,...,...
5226,현재상태/증상지속,한숨 자고 일어나면 괜찮으려나? 해서 자고 일어났는데도 똑같아요.,NaN,358
5227,현재상태/증상지속,자고 나면 괜찮을 줄 알았는데 비슷해…,NaN,358
5228,현재상태/증상지속,지금도 상태가 계속 안 좋아.,NaN,358
5229,현재상태/증상지속,이게 제 마음과 상관없이 증상이 계속 나타나요.,NaN,358


In [13]:
#챗봇 결측값있는 행 전체 제거

new_chatbot_data = chatbot_data.dropna(axis=0)
new_chatbot_data

,구분,유저,챗봇,category
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,0
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,0
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",0
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,1
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,1
...,...,...,...,...
5196,치료이력/응급실,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?,355
5197,치료이력/응급실,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.,355
5213,현재상태/증상악화,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.,357
5214,현재상태/증상악화,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?,357


In [14]:
len(new_chatbot_data)

1034

In [15]:
question, answer=list(new_chatbot_data['유저']), list(new_chatbot_data['챗봇'])

In [16]:
len(question)

1034

In [17]:
question = question[:100]
answer = answer[:100]

In [18]:
for i in range(10):
    print('Q : ' + question[i])
    print('A : ' + answer[i])
    print()


Q : 제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.
A : 감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.

Q : 더 이상 내 감정을 내가 컨트롤 못 하겠어.
A : 저도 그 기분 이해해요. 많이 힘드시죠?

Q : 하루종일 오르락내리락 롤러코스터 타는 기분이에요.
A : 그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.

Q : 평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되면 폭발해버려요.
A : 화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.

Q : 예전보다 화내는 게 과격해진 거 같아.
A : 정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.

Q : 화가 안 참아져.
A : 화가 너무 많이 날 때는 심호흡을 해보는 게 어떨까요? 씁- 후-

Q : 나도 그런 거 아닌가 걱정돼.
A : 걱정이 많으시군요. 걱정거리를 정리하는 시간을 가지면 도움이 되지 않을까요?  

Q : 처음엔 치료 받으면 낫겠지 생각했는데, 낫지도 않고 계속 병원을 다니니까 너무 걱정되요.
A : 당연히 걱정이 되는 상황인 것 같아요. 저도 마음이 아프네요.

Q : 수술한다는 말에 얼마나 걱정이 되던지…
A : 모든 문제는 해결되기 마련이잖아요. 마음을 편히 드세요.

Q : 평생 이렇게 살아야할까봐 걱정이야.
A : 건강 때문에 걱정이 많으시군요. 잘 관리하고 계시다면 큰 문제는 없을 거예요.



In [20]:
# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()…]")

In [21]:
def pos_tag(sentences):
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    # 문장 품사 변수 초기화
    sentences_pos = []
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", sentence)

        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)

    return sentences_pos


In [22]:
#각변수에 저장
question = pos_tag(question)
answer = pos_tag(answer)

In [23]:
question

['제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요',
 '더 이상 내 감정 을 내 가 컨트롤 못 하겠어',
 '하루 종일 오르락내리락 롤러코스터 타는 기분 이에요',
 '평소 다른 일 을 할 때 도 비슷해요 생각 한 대로 안되면 화가 나고 그런 상황 이 지속 되면 폭발 해 버려요',
 '예전 보다 화 내는 게 과격해진 거 같아',
 '화가 안 참아져',
 '나도 그런 거 아닌가 걱정 돼',
 '처음 엔 치료 받으면 낫겠지 생각 했는데 낫지도 않고 계속 병원 을 다니니까 너무 걱정 되요',
 '수술 한다는 말 에 얼마나 걱정 이 되던지',
 '평생 이렇게 살 아 야할까봐 걱정 이야',
 '나도 혈압 이 높은 편 인데 요즘 몸 도 안 좋고 해서 자꾸 걱정 이 되네',
 '나도 이러기 싫은데 자꾸 건강 을 걱정 하게 돼',
 '위염 이 위암 이나 다른 암 으로 발전 하면 어떡하지 이런 생각 을 많이 하는 것 같아요',
 '괜히 진짜 큰 병 이 면 어쩌나 싶기도 하고 별거 아닌 걸 로 병원 가는 것 도 좀 그렇 기도 하고',
 '수술 도 잘 됐고 의사 도 회복 만 잘 하면 된다는데 자꾸 걱정 이 돼요',
 '학자금 은 언제 다 갚나 아직도 몇 년 은 더 남았는데',
 '경제 적 인 문제 만 좀 풀려주면 좋겠는데 쉽지가 않네요',
 '언제 쯤 저 는 돈 걱정 없이 살 수 있을까요',
 '내 가 이 회사 를 다니는 게 맞나 그런 생각 도 들고요',
 '앞 으로 집은 어떻게 사나 싶어요',
 '좋은 환경 에 있어도 잘 될까 말까 하는데 이런 환경 에서 사람 답 게 살 수 있을까',
 '걱정 이 돼서 밤 에 잠 이 안 와',
 '또 못 자면 어떡하지 하고 걱정 이 드는데 그거 때문 에 더 못 자는 거 같아요',
 '언제 또 이렇게 못 잘 지 모르겠고 또 못 자게 될까봐 그게 걱정 돼',
 '위염 인 줄 알았는데 암 이었다는 사람 도 있었고 걱정 이 되지',
 '간암 도 초기 에는 잘 모른다고 하니까 내 가 간암 일 수도 있지 않나 내

In [24]:
answer

['감정 이 조절 이 안 될 때 만큼 힘들 때 는 없는 거 같아요',
 '저 도 그 기분 이해해 요 많이 힘드시죠',
 '그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요',
 '화가 폭발 할 것 같을 때 는 그 자리 를 피하 는 것 도 좋은 방법 이라고 생각 해요',
 '정말 힘드시겠어요 화 는 남 에게도 스스로 에게도 상처 를 주잖아요',
 '화가 너무 많이 날 때 는 심호흡 을 해보는 게 어떨까 요 씁 - 후 -',
 '걱정 이 많으시군요 걱정거리 를 정리 하는 시간 을 가지 면 도움 이 되지 않을까요',
 '당연히 걱정 이 되는 상황 인 것 같아요 저 도 마음 이 아프네요',
 '모든 문제 는 해결 되기 마련 이 잖아요 마음 을 편히 드세요',
 '건강 때문 에 걱정 이 많으시군요 잘 관리 하고 계시다면 큰 문제 는 없을 거 예요',
 '너무 걱정 하지 마세요 현 대 의학 을 믿자구요',
 '걱정 이 없던 병 도 만들 수 있어요 머리 를 비울 수 있게 영화 를 한 편 보는 건 어때요',
 '걱정 이 많이 될 거 같아요 한번 병원 에 가서 진찰 을 받아 보는 건 어때요',
 '저 도 그 마음 이해해 요 하지만 긍정 적 으로 한번 생각 해보자구요',
 '건강 은 늘 걱정 되는 부분 인 것 같아요',
 '돈 문제 로 걱정 이 많으시군요 그럴 때 는 정말 막막한 것 같아요',
 '이럴 때 복권 이라도 당첨 되면 좋을 텐데요',
 '마음 이 힘드시겠어요 돈 걱정 없는 세상 이 얼른 왔으면 좋겠어요',
 '걱정 이 되는 그 마음 이해해 요 멋진 미래 를 꿈꾸기에는 세상 이 너무 팍팍한 것 같아요',
 '언젠가 는 나아질 거 라는 믿음 이 사람 을 버티게 만들어주는 것 같아요',
 '계속 앞 으로 가다 보면 당신 에게 맞는 길도 찾을 수 있지 않을까요',
 '걱정 을 하는 시간 을 정해 놓는 것 도 좋은 방법 이라고 들었어요',
 '잠 은 중요하죠 걱정 은 잠시 내려놓으세요',
 '눈 을 감고 아무 생각 도 하지 않는 거 예요 그러면 곧 

In [27]:
# 형태소분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('User : ' + question[i])
    print('ASHP : ' + answer[i])
    print()

User : 제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요
ASHP : 감정 이 조절 이 안 될 때 만큼 힘들 때 는 없는 거 같아요

User : 더 이상 내 감정 을 내 가 컨트롤 못 하겠어
ASHP : 저 도 그 기분 이해해 요 많이 힘드시죠

User : 하루 종일 오르락내리락 롤러코스터 타는 기분 이에요
ASHP : 그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요

User : 평소 다른 일 을 할 때 도 비슷해요 생각 한 대로 안되면 화가 나고 그런 상황 이 지속 되면 폭발 해 버려요
ASHP : 화가 폭발 할 것 같을 때 는 그 자리 를 피하 는 것 도 좋은 방법 이라고 생각 해요

User : 예전 보다 화 내는 게 과격해진 거 같아
ASHP : 정말 힘드시겠어요 화 는 남 에게도 스스로 에게도 상처 를 주잖아요

User : 화가 안 참아져
ASHP : 화가 너무 많이 날 때 는 심호흡 을 해보는 게 어떨까 요 씁 - 후 -

User : 나도 그런 거 아닌가 걱정 돼
ASHP : 걱정 이 많으시군요 걱정거리 를 정리 하는 시간 을 가지 면 도움 이 되지 않을까요

User : 처음 엔 치료 받으면 낫겠지 생각 했는데 낫지도 않고 계속 병원 을 다니니까 너무 걱정 되요
ASHP : 당연히 걱정 이 되는 상황 인 것 같아요 저 도 마음 이 아프네요

User : 수술 한다는 말 에 얼마나 걱정 이 되던지
ASHP : 모든 문제 는 해결 되기 마련 이 잖아요 마음 을 편히 드세요

User : 평생 이렇게 살 아 야할까봐 걱정 이야
ASHP : 건강 때문 에 걱정 이 많으시군요 잘 관리 하고 계시다면 큰 문제 는 없을 거 예요



In [28]:
# 질문과 대답 문장들을 하나로 합침
sentences = []
sentences.extend(question)

In [29]:
sentences

['제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요',
 '더 이상 내 감정 을 내 가 컨트롤 못 하겠어',
 '하루 종일 오르락내리락 롤러코스터 타는 기분 이에요',
 '평소 다른 일 을 할 때 도 비슷해요 생각 한 대로 안되면 화가 나고 그런 상황 이 지속 되면 폭발 해 버려요',
 '예전 보다 화 내는 게 과격해진 거 같아',
 '화가 안 참아져',
 '나도 그런 거 아닌가 걱정 돼',
 '처음 엔 치료 받으면 낫겠지 생각 했는데 낫지도 않고 계속 병원 을 다니니까 너무 걱정 되요',
 '수술 한다는 말 에 얼마나 걱정 이 되던지',
 '평생 이렇게 살 아 야할까봐 걱정 이야',
 '나도 혈압 이 높은 편 인데 요즘 몸 도 안 좋고 해서 자꾸 걱정 이 되네',
 '나도 이러기 싫은데 자꾸 건강 을 걱정 하게 돼',
 '위염 이 위암 이나 다른 암 으로 발전 하면 어떡하지 이런 생각 을 많이 하는 것 같아요',
 '괜히 진짜 큰 병 이 면 어쩌나 싶기도 하고 별거 아닌 걸 로 병원 가는 것 도 좀 그렇 기도 하고',
 '수술 도 잘 됐고 의사 도 회복 만 잘 하면 된다는데 자꾸 걱정 이 돼요',
 '학자금 은 언제 다 갚나 아직도 몇 년 은 더 남았는데',
 '경제 적 인 문제 만 좀 풀려주면 좋겠는데 쉽지가 않네요',
 '언제 쯤 저 는 돈 걱정 없이 살 수 있을까요',
 '내 가 이 회사 를 다니는 게 맞나 그런 생각 도 들고요',
 '앞 으로 집은 어떻게 사나 싶어요',
 '좋은 환경 에 있어도 잘 될까 말까 하는데 이런 환경 에서 사람 답 게 살 수 있을까',
 '걱정 이 돼서 밤 에 잠 이 안 와',
 '또 못 자면 어떡하지 하고 걱정 이 드는데 그거 때문 에 더 못 자는 거 같아요',
 '언제 또 이렇게 못 잘 지 모르겠고 또 못 자게 될까봐 그게 걱정 돼',
 '위염 인 줄 알았는데 암 이었다는 사람 도 있었고 걱정 이 되지',
 '간암 도 초기 에는 잘 모른다고 하니까 내 가 간암 일 수도 있지 않나 내

In [30]:
sentences.extend(answer)

In [31]:
sentences

['제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요',
 '더 이상 내 감정 을 내 가 컨트롤 못 하겠어',
 '하루 종일 오르락내리락 롤러코스터 타는 기분 이에요',
 '평소 다른 일 을 할 때 도 비슷해요 생각 한 대로 안되면 화가 나고 그런 상황 이 지속 되면 폭발 해 버려요',
 '예전 보다 화 내는 게 과격해진 거 같아',
 '화가 안 참아져',
 '나도 그런 거 아닌가 걱정 돼',
 '처음 엔 치료 받으면 낫겠지 생각 했는데 낫지도 않고 계속 병원 을 다니니까 너무 걱정 되요',
 '수술 한다는 말 에 얼마나 걱정 이 되던지',
 '평생 이렇게 살 아 야할까봐 걱정 이야',
 '나도 혈압 이 높은 편 인데 요즘 몸 도 안 좋고 해서 자꾸 걱정 이 되네',
 '나도 이러기 싫은데 자꾸 건강 을 걱정 하게 돼',
 '위염 이 위암 이나 다른 암 으로 발전 하면 어떡하지 이런 생각 을 많이 하는 것 같아요',
 '괜히 진짜 큰 병 이 면 어쩌나 싶기도 하고 별거 아닌 걸 로 병원 가는 것 도 좀 그렇 기도 하고',
 '수술 도 잘 됐고 의사 도 회복 만 잘 하면 된다는데 자꾸 걱정 이 돼요',
 '학자금 은 언제 다 갚나 아직도 몇 년 은 더 남았는데',
 '경제 적 인 문제 만 좀 풀려주면 좋겠는데 쉽지가 않네요',
 '언제 쯤 저 는 돈 걱정 없이 살 수 있을까요',
 '내 가 이 회사 를 다니는 게 맞나 그런 생각 도 들고요',
 '앞 으로 집은 어떻게 사나 싶어요',
 '좋은 환경 에 있어도 잘 될까 말까 하는데 이런 환경 에서 사람 답 게 살 수 있을까',
 '걱정 이 돼서 밤 에 잠 이 안 와',
 '또 못 자면 어떡하지 하고 걱정 이 드는데 그거 때문 에 더 못 자는 거 같아요',
 '언제 또 이렇게 못 잘 지 모르겠고 또 못 자게 될까봐 그게 걱정 돼',
 '위염 인 줄 알았는데 암 이었다는 사람 도 있었고 걱정 이 되지',
 '간암 도 초기 에는 잘 모른다고 하니까 내 가 간암 일 수도 있지 않나 내

In [44]:
len(max(sentences))

48

In [32]:
words = []

In [33]:
for sentence in sentences:
  for word in sentence.split(): #공백문자 기준으로 word 저장
    words.append(word)

#길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

#중복된 단어는 삭제
words = list(set(words))


In [34]:
words

['아니면',
 '-',
 '관심',
 '이었다는',
 '않을지도',
 '시내',
 '였어요',
 '가져가도록',
 '쯤',
 '할게요',
 '이제',
 '생긴',
 '차라리',
 '가지',
 '세게',
 '되기',
 '같아',
 '달려들까봐',
 '얼얼하더라',
 '회복',
 '그랬군요',
 '않고',
 '남',
 '좋을',
 '그렇지',
 '얘기',
 '선생님',
 '잡고',
 '당첨',
 '먹는',
 '한국',
 '하려고',
 '겪어',
 '아파요',
 '이라고',
 '주고',
 '이런',
 '사람',
 '치료',
 '아들',
 '타면',
 '머리',
 '이었던',
 '들고',
 '즐겁지가',
 '지르고',
 '자녀',
 '오면',
 '좋겠는데',
 '가져가',
 '비둘기',
 '회사',
 '살',
 '걸',
 '휴식',
 '최대한',
 '거',
 '낫지도',
 '발견',
 '비밀',
 '피하',
 '경제',
 '말까',
 '나쁜',
 '곳',
 '눈',
 '아무렇지',
 '되고',
 '잤는지',
 '그러면',
 '빼앗기는',
 '높은',
 '봐도',
 '상태',
 '믿자구요',
 '것',
 '얼떨떨했던',
 '평소',
 '같을',
 '가장',
 '문제',
 '좋겠다고',
 '모르겠어요',
 '사고',
 '왔으면',
 '옆',
 '과격해진',
 '정말',
 '가본',
 '버려져',
 '의',
 '비슷해요',
 '튀어',
 '하루',
 '익숙한',
 '완치',
 '스럽',
 '어때요',
 '예요',
 '위염',
 '엔',
 '하고',
 '꽤',
 '폭발',
 '꼭',
 '걱정',
 '싶기도',
 '여',
 '여기',
 '없으니까',
 '다해',
 '증거',
 '수술',
 '아닌가',
 '런가',
 '마세요',
 '찾을',
 '부리',
 '좋고',
 '만일',
 '늘',
 '언젠가',
 '같아요',
 '살았는데',
 '느낌',
 '조절',
 '만들',
 '놀라고',
 '또',
 '스러운',
 '걸어',
 '가까이',
 '자체',
 '아직도',
 '좋아질까',

In [35]:
PAD = "<PADDING>"   # 패딩
STA = "<START>"     # 시작
END = "<END>"       # 끝
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

In [36]:
# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

In [37]:
words

['<PADDING>',
 '<START>',
 '<END>',
 '<OOV>',
 '아니면',
 '-',
 '관심',
 '이었다는',
 '않을지도',
 '시내',
 '였어요',
 '가져가도록',
 '쯤',
 '할게요',
 '이제',
 '생긴',
 '차라리',
 '가지',
 '세게',
 '되기',
 '같아',
 '달려들까봐',
 '얼얼하더라',
 '회복',
 '그랬군요',
 '않고',
 '남',
 '좋을',
 '그렇지',
 '얘기',
 '선생님',
 '잡고',
 '당첨',
 '먹는',
 '한국',
 '하려고',
 '겪어',
 '아파요',
 '이라고',
 '주고',
 '이런',
 '사람',
 '치료',
 '아들',
 '타면',
 '머리',
 '이었던',
 '들고',
 '즐겁지가',
 '지르고',
 '자녀',
 '오면',
 '좋겠는데',
 '가져가',
 '비둘기',
 '회사',
 '살',
 '걸',
 '휴식',
 '최대한',
 '거',
 '낫지도',
 '발견',
 '비밀',
 '피하',
 '경제',
 '말까',
 '나쁜',
 '곳',
 '눈',
 '아무렇지',
 '되고',
 '잤는지',
 '그러면',
 '빼앗기는',
 '높은',
 '봐도',
 '상태',
 '믿자구요',
 '것',
 '얼떨떨했던',
 '평소',
 '같을',
 '가장',
 '문제',
 '좋겠다고',
 '모르겠어요',
 '사고',
 '왔으면',
 '옆',
 '과격해진',
 '정말',
 '가본',
 '버려져',
 '의',
 '비슷해요',
 '튀어',
 '하루',
 '익숙한',
 '완치',
 '스럽',
 '어때요',
 '예요',
 '위염',
 '엔',
 '하고',
 '꽤',
 '폭발',
 '꼭',
 '걱정',
 '싶기도',
 '여',
 '여기',
 '없으니까',
 '다해',
 '증거',
 '수술',
 '아닌가',
 '런가',
 '마세요',
 '찾을',
 '부리',
 '좋고',
 '만일',
 '늘',
 '언젠가',
 '같아요',
 '살았는데',
 '느낌',
 '조절',
 '만들',
 '놀라고',
 '또',


In [38]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)} 
index_to_word = {index: word for index, word in enumerate(words)}

In [39]:
index_to_word

{0: '<PADDING>',
 1: '<START>',
 2: '<END>',
 3: '<OOV>',
 4: '아니면',
 5: '-',
 6: '관심',
 7: '이었다는',
 8: '않을지도',
 9: '시내',
 10: '였어요',
 11: '가져가도록',
 12: '쯤',
 13: '할게요',
 14: '이제',
 15: '생긴',
 16: '차라리',
 17: '가지',
 18: '세게',
 19: '되기',
 20: '같아',
 21: '달려들까봐',
 22: '얼얼하더라',
 23: '회복',
 24: '그랬군요',
 25: '않고',
 26: '남',
 27: '좋을',
 28: '그렇지',
 29: '얘기',
 30: '선생님',
 31: '잡고',
 32: '당첨',
 33: '먹는',
 34: '한국',
 35: '하려고',
 36: '겪어',
 37: '아파요',
 38: '이라고',
 39: '주고',
 40: '이런',
 41: '사람',
 42: '치료',
 43: '아들',
 44: '타면',
 45: '머리',
 46: '이었던',
 47: '들고',
 48: '즐겁지가',
 49: '지르고',
 50: '자녀',
 51: '오면',
 52: '좋겠는데',
 53: '가져가',
 54: '비둘기',
 55: '회사',
 56: '살',
 57: '걸',
 58: '휴식',
 59: '최대한',
 60: '거',
 61: '낫지도',
 62: '발견',
 63: '비밀',
 64: '피하',
 65: '경제',
 66: '말까',
 67: '나쁜',
 68: '곳',
 69: '눈',
 70: '아무렇지',
 71: '되고',
 72: '잤는지',
 73: '그러면',
 74: '빼앗기는',
 75: '높은',
 76: '봐도',
 77: '상태',
 78: '믿자구요',
 79: '것',
 80: '얼떨떨했던',
 81: '평소',
 82: '같을',
 83: '가장',
 84: '문제',
 85: '좋겠다고',
 86: '모르

In [40]:
question[0]

'제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요'

In [45]:
max_sequences = 50

In [46]:
# 데이터 타입
ENCODER_INPUT  = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

In [47]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type):
  sentences_index = [] #문장 -> 단어 -> 인덱스 -> 저장

  # 모든 문장에 대해서 반복
  for sentence in sentences:
    sentence_index = [] #각각의 문장을 구성하는 단어들에 대한 index가 저장되는 리스트 변수
    
    # 디코더 입력일 경우 맨 앞에 START 태그 추가
    if type == DECODER_INPUT:
      sentence_index.extend([vocabulary[STA]])

    # 문장의 단어들을 띄어쓰기로 분리
    for word in sentence.split(): #[12시, 땡]
      if vocabulary.get(word) is not None: #파이썬 문법이 딕셔너리는 키에대한 값이 없으면 none이 나온다.
        # 사전에 있는 단어면 해당 인덱스를 추가
        sentence_index.extend([vocabulary[word]])
      else: # 사전에 없는 단어면 OOV 인덱스를 추가
        sentence_index.extend([vocabulary[OOV]])

    # 최대 길이 검사
    if type == DECODER_TARGET:
      # 디코더 목표일 경우 맨 뒤에 END 태그 추가
      if len(sentence_index) >= max_sequences:
        sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
      else:
        sentence_index += [vocabulary[END]]
    else:
       if len(sentence_index) > max_sequences:
         sentence_index = sentence_index[:max_sequences]
            
    # 최대 길이에 없는 공간은 패딩 인덱스로 채움
    sentence_index += (max_sequences - len(sentence_index)) * [vocabulary[PAD]]

    sentences_index.append(sentence_index)

  return np.array(sentences_index)

In [48]:
#인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

#첫 번째 인코더 입력 출력 ("제 감정 이 이상해진 것 같아요 남편 만 보면 화가 치밀어 오르고 감정 조절 이 안되요")
x_encoder[0]

array([426, 443, 489, 571,  79, 126, 197, 866, 342, 669, 788, 656, 443,
       129, 489, 354,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [49]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫 번째 디코더 입력 출력 (START 블라블라)
x_decoder[0]

array([  1, 443, 489, 129, 489, 736, 601, 430, 717, 351, 430, 652, 620,
        60, 126,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [50]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer, word_to_index, DECODER_TARGET)

# 첫 번째 디코더 목표 출력 (블라블라 END)
y_decoder[0]


array([443, 489, 129, 489, 736, 601, 430, 717, 351, 430, 652, 620,  60,
       126,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [51]:
np.shape(x_encoder)

(100, 50)

In [52]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words)))


In [54]:
one_hot_data.shape #(100, 50, 880)
y_decoder.shape #(100,50)

(100, 50)

In [55]:
for i, sequence in enumerate(y_decoder):
  print(i,sequence)
  
  #각 단어를 880차원 원핫벡터로 표현

0 [443 489 129 489 736 601 430 717 351 430 652 620  60 126   2   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
1 [158 452 671 343 579 610 352 365   2   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
2 [556 430 652 753 277 879 222 176 277 879  72 561 314  79 452 362   2   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
3 [669 107 148  79  82 430 652 671 877 609  64 652  79 452 394 677  38 833
 326   2   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
4 [ 91 721 309 652  26 577 298 577 383 609 177   2   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0

In [56]:
# 디코더 목표를 원핫인코딩으로 변환
# 학습시 입력은 인덱스이지만, 출력은 원핫인코딩 형식임
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i, j, index] = 1 #원핫 인코딩
        #i는 100개 문장, j는 30개 단어, index는 454개 중 1개를 1로 셋팅

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫 번째 디코더 목표 출력
y_decoder[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [57]:
# 임베딩 벡터 차원
embedding_dim = 100
# LSTM 히든레이어 차원
lstm_hidden_dim = 128

In [58]:
#훈련 모델 인코더 정의

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim)(encoder_inputs)

# return_state가 True면 상태값 리턴
# LSTM은 state_h(hidden state)와 state_c(cell state) 2개의 상태 존재
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim,
                                                dropout=0.1,
                                                recurrent_dropout=0.5,
                                                return_state=True)(encoder_outputs)

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]



In [59]:
#훈련 모델 디코더 정의

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,))

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words), embedding_dim)
decoder_outputs = decoder_embedding(decoder_inputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임 스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리하기 위함
decoder_lstm = layers.LSTM(lstm_hidden_dim,
                           dropout=0.1,
                           recurrent_dropout=0.5,
                           return_state=True,
                           return_sequences=True)

# initial_state를 인코더의 상태로 초기화
decoder_outputs, _, _ = decoder_lstm(decoder_outputs,
                                     initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [60]:
# 훈련 모델 정의
#--------------------------------------------

# 입력과 출력으로 함수형 API 모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
encoder_model = models.Model(encoder_inputs, encoder_states)

In [62]:
#예측 모델 디코더 정의
#--------------------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,))
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]    

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs)

# LSTM 레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs,
                                                 initial_state=decoder_states_inputs)

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense 레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs] + decoder_states_inputs,
                      [decoder_outputs] + decoder_states)

In [63]:
# 인덱스를 문장으로 변환
def convert_index_to_text(indexs, vocabulary): 
    
    sentence = ''
    
    # 모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            # 종료 인덱스면 중지
            break;
        if vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            # 사전에 없는 인덱스면 OOV 단어를 추가
            sentence.extend([vocabulary[OOV_INDEX]])
            
        # 빈칸 추가
        sentence += ' '

    return sentence

In [64]:
# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

In [65]:
# 에폭 반복
for epoch in range(20):
    print('Total Epoch :', epoch + 1)

    # 훈련 시작
    history = model.fit([x_encoder, x_decoder],
                        y_decoder,
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy :', history.history['accuracy'][-1])
    print('loss :', history.history['loss'][-1])
    
    # 문장 예측 테스트
    # (하루 종일 오르락내리락 롤러코스터 타는 기분 이에요) -> (그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요)
    input_encoder = x_encoder[2].reshape(1, x_encoder[2].shape[0])
    input_decoder = x_decoder[2].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])
    
    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0], 1) 
    
    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch : 1
accuracy : 0.7699999809265137
loss : 1.1513625383377075
저 이 이 이 이 때 하는 을 을 것 수 것 것 거 

Total Epoch : 2
accuracy : 0.8370000123977661
loss : 0.7495766282081604
저 는 는 마음 때 때 먹었는지 때 을 잘 잤는지 체크 도움 것 도 좋아요 

Total Epoch : 3
accuracy : 0.9282000064849854
loss : 0.41658905148506165
걱정 에 는 밥 는 많아요 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 

Total Epoch : 4
accuracy : 0.9667999744415283
loss : 0.19530105590820312
걱정 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 

Total Epoch : 5
accuracy : 0.9801999926567078
loss : 0.09550466388463974
걱정 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 

Total Epoch : 6
accuracy : 0.9846000075340271
loss : 0.05995303764939308
아무 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 

Total Epoch : 7
accuracy : 0.9879999756813049
loss : 0.04310605674982071
그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 

Total Epoch : 8
accuracy : 0.9941999912261963
loss : 0.02538727968931198
그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 

Total Epoch : 9
accuracy : 0.996399998664856
loss : 0.01661573909223079

In [66]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences)
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT)
    
    return input_seq

In [67]:
# 텍스트 생성
def generate_text(input_seq):
    #array([[156, 414,  89, 405, 280, 158,   0,   0,   0,   0,   0,   0,   0,
    #      0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
    #      0,   0,   0,   0]])
    
    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq)

    # 목표 시퀀스 초기화
    target_seq = np.zeros((1, 1))
    
    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0, 0] = STA_INDEX
    
    # 인덱스 초기화
    indexs = []
    
    # 디코더 타임 스텝 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict(
                                                [target_seq] + states)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0, 0, :])
        indexs.append(index)
        
        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break

        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = index
        
        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c]

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
        
    return sentence

In [69]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('하루 종일 오르락내리락 롤러코스터 타는 기분 이에요')
input_seq

array([[ 97, 409, 700, 834, 253, 343, 727,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [70]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'그럴 때 는 밥 은 잘 먹었는지 잠 은 잘 잤는지 체크 해보는 것 도 좋아요 '

In [71]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('오늘 기분이 꿀꿀해')
input_seq

array([[418, 343, 489,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [72]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'안 좋은 상황 이신 거 같아요 무슨 일인 지 저 에게 자세히 얘기 해줄 수 있을까요 '

In [73]:
input_seq = make_predict_input('코로나 진짜 짜증나')
input_seq

array([[  3, 574,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [74]:
sentence = generate_text(input_seq)
sentence

'막상 그 상황 이 닥치 면 아무렇지 않을지도 몰라요 너무 걱정 마세요 '

In [75]:
input_seq = make_predict_input('오늘 하는 것마다 다 잘 돼 너무 좋아')
input_seq

array([[418, 643,  79, 459, 507, 879, 196, 224,   3,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [76]:
sentence = generate_text(input_seq)
sentence

'너무 두렵고 무섭다면 굳이 이겨내려고 하지 않아도 돼요 한국 은 대중교통 이 편리하잖아요 '

In [77]:
input_seq = make_predict_input('오늘 하는 것마다 다 안돼 너무 슬퍼')
input_seq

array([[418, 643,  79, 459, 507, 736, 196, 224,   3,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0]])

In [78]:
sentence = generate_text(input_seq)
sentence

'너무 두렵고 무섭다면 굳이 이겨내려고 하지 않아도 돼요 한국 은 대중교통 이 편리하잖아요 '